In [13]:
import numpy as np
import os  # for mkdir
import sys
import math
import librosa # conda install -c conda-forge librosa
import soundfile as sf

##-----------   paramManager  interface   ------------------##
# git clone https://github.com/prashanthtr/paramManager
from parammanager import paramManager
from sonyganformat import sonyGanJson

### Chunk file into 1 seconds and based on SampleRate

In [14]:
chunkDuration = 1; # 1 sec

In [22]:
from utils import longSilence

datapath='srcwav'  #root folder
fileList = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f)) and (os.path.splitext(f)[1]==".wav" or os.path.splitext(f)[1]==".aif" or os.path.splitext(f)[1]==".aiff")]
#stub: fileList = ["silence.wav"]


createSegments = True

if createSegments == True:
    for fileName in fileList:

        sig,sr=librosa.core.load(datapath + "/" + fileName, sr=None)
        Fs = 22050

        # resample
        sig= librosa.core.resample(sig, sr, 22050) 

        fileBase = os.path.splitext(fileName)[0]

        numSamplesPerChunk = chunkDuration*Fs;
        numChunks = math.floor(len(sig)/numSamplesPerChunk)
        segment = []

        mean = np.mean(sig)

        for i in range(numChunks-1):
            segment = sig[i*numSamplesPerChunk:(i+1)*numSamplesPerChunk]
            mean = np.mean(segment)
            x = [sample-mean for sample in segment] #dc filter
            if( longSilence(x, 0.01) ):
                print(fileBase + " ",  i)
                norm = librosa.util.normalize(x)
                sf.write("segmented" + "/" + fileBase + "--" + "v-" + str(i) + ".wav", norm, sr)
            else:
                print("Discarding a silent segment ", i)
            i = i + 1;

max 128
arp_1  0
max 7
arp_1  1
max 5
arp_1  2
max 7
arp_1  3
max 6
arp_1  4
max 7
arp_1  5
max 5
arp_1  6
max 7
arp_1  7
max 6
arp_1  8
max 5
arp_1  9
max 6
arp_1  10
max 5
arp_1  11
max 6
arp_1  12
max 5
arp_1  13
max 5
arp_1  14
max 6
arp_1  15
max 5
arp_1  16
max 7
arp_1  17
max 5
arp_1  18
max 6
arp_1  19
max 6
arp_1  20
max 5
arp_1  21
max 5
arp_1  22
max 5
arp_1  23
max 6
arp_1  24
max 4
arp_1  25
max 4
arp_1  26
max 6
arp_1  27
max 5
arp_1  28
max 6
arp_1  29
max 5
arp_1  30
max 5
arp_1  31
max 5
arp_1  32
max 7
arp_1  33
max 6
arp_1  34
max 9
arp_1  35
max 6
arp_1  36
max 5
arp_1  37
max 7
arp_1  38
max 7
arp_1  39
max 6
arp_1  40
max 7
arp_1  41
max 6
arp_1  42
max 6
arp_1  43
max 4
arp_1  44
max 5
arp_1  45
max 5
arp_1  46
max 7
arp_1  47
max 5
arp_1  48
max 6
arp_1  49
max 4
arp_1  50
max 4
arp_1  51
max 6
arp_1  52
max 6
arp_1  53
max 5
arp_1  54
max 5
arp_1  55
max 6
arp_1  56
max 5
arp_1  57
max 6
arp_1  58
max 6
arp_1  59
max 4
arp_1  60
max 6
arp_1  61
max 5
arp_1  62


### Generate record formats (parammanager, sonyGan, tfrecords)

In [23]:
#Setup paths for paramManager and examplesJSON
datapath = 'segmented'
parampath = 'segmented'

### ParamManager

In [28]:
pm=paramManager.paramManager(datapath,parampath) #filebase is directory name
pm.initParamFiles(overwrite=True) ##-----------   paramManager  interface ------------------##
pm.checkIntegrity()

paramList = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(parampath, os.path.splitext(f)[0] + ".params"))]
for p in paramList:
    soundText = os.path.splitext(p)[0].split("--")
    foo=pm.getParams(datapath + '/' + p) #extension is optional
    pm.addMetaParam(parampath + '/' + p,"soundName", soundText[0])
    pm.addMetaParam(parampath + '/' + p,"segmentNum", int(soundText[len(soundText)-1].split("-")[1]))

In [29]:
paramList = ["arp_1--v-3.params"]

for p in paramList:
    foo=pm.getParams(parampath + '/' + p) #extension is optional
    print(foo)

{'meta': {'filename': 'arp_1--v-3.wav', 'soundName': 'arp_1', 'segmentNum': 3}}


### SonyGan format

In [30]:
# Read all files in datapath and create a examples.json
print("parameters")
sg = sonyGanJson.SonyGanJson(datapath,"natural",22050,"O'REILLY")

parameters


### Writing records in single SG json file

In [31]:
#sg.ppRecords() # pretty prints in json format
sg.write2File("sonyGan.json") 

written output to  sonyGan.json
